## Kubernetes Agent

Also, the instructions are changes from [basic use case](./basic_tool_use_case.ipynb), and now we are force to the LLM to use kube_client when some Kubernetes keyword appears in the prompt. Take a look to these instructions and take a time to understand the implications they can have to the LLM.

We’re going to follow the same approach as in the basic tool use case, using Llama Stack and MCP. However, this time, we’ll explore a more realistic scenario. We’ve implemented a Kubernetes client in [kube_client.py](./tools/kube_client.py), which allows us to connect to a Kubernetes cluster and execute commands.

To simplify the use case, we’ll use a local Minikube instance, which we are already logged into.

Additionally, the model instructions differ from those in the basic use case. In this case, we explicitly require the LLM to use kube_client whenever Kubernetes-related keywords appear in the prompt.
Take a moment to review these instructions and understand the implications they may have on the model’s behavior.

In [ ]:
import os

from llama_stack_client import Agent, AgentEventLogger
from llama_stack.distribution.library_client import LlamaStackAsLibraryClient
from llama_stack_client.types.toolgroup_register_params import McpEndpoint


model = "llama3.2:3b-instruct-fp16"
os.environ["INFERENCE_MODEL"] = model
client = LlamaStackAsLibraryClient('ollama')
client.initialize()


client.toolgroups.register(
    toolgroup_id="mcp::custom",
    provider_id="model-context-protocol",
    mcp_endpoint=McpEndpoint(uri="http://127.0.0.1:5000/sse"),
)


instructions = """
You are a smart assistant that helps users with his questions.
You have differents available tools and you must use to retrieve information or perform actions.
You must not generate code or scripts.
No simulate the output, perform all operations with tools available.
You are force to use kube_command tool to create resources in kubernetes, so if the user ask create something you MUST use the tool.
Not use yaml to create resources, Just use kubectl commands
You have to add kubectl in the commands you pass to kubec_command tool, for instance kubectl get pods 
"""

agent = Agent(
  client,
  model=model,
  instructions=instructions,
  tools=["mcp::custom"],
  sampling_params={
    "strategy": {"type": "top_p", "temperature": 0.2, "top_p": 0.9},
  },
)
session_id = agent.create_session('new_session')

/home/jbarea/Documents/hcm/ai/ai-lab-playground/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using config ollama:

apis:
- agents
- datasetio
- eval
- inference
- post_training
- safety
- scoring
- telemetry
- tool_runtime
- vector_io
benchmarks: []
container_image: null
datasets: []
external_providers_dir: null
image_name: ollama
inference_store:
  db_path: /home/jbarea/.llama/distributions/ollama/inference_store.db
  type: sqlite
logging: null
metadata_store:
  db_path: /home/jbarea/.llama/distributions/ollama/registry.db
  namespace: null
  type: sqlite
models:
- metadata: {}
  model_id: llama3.2:3b-instruct-fp16
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: ollama
  provider_model_id: null
- metadata:
    embedding_dimension: 384
  model_id: all-MiniLM-L6-v2
  model_type: !!python/object/apply:llama_stack.apis.models.models.ModelType
  - embedding
  provider_id: ollama
  provider_model_id: all-minilm:latest
providers:
  agents:
  - config:
      persistence_store:
        db_path: /home/jbarea/.llama/distributions/ollama/agents_store.db
        namespace: null
        type: sqlite
      responses_store:
        db_path: /home/jbarea/.llama/distributions/ollama/responses_store.db
        type: sqlite
    provider_id: meta-reference
    provider_type: inline::meta-reference
  datasetio:
  - config:
      kvstore:
        db_path: /home/jbarea/.llama/distributions/ollama/huggingface_datasetio.db
        namespace: null
        type: sqlite
    provider_id: huggingface
    provider_type: remote::huggingface
  - config:
      kvstore:
        db_path: /home/jbarea/.llama/distributions/ollama/localfs_datasetio.db
        namespace: null
        type: sqlite
    provider_id: localfs
    provider_type: inline::localfs
  eval:
  - config:
      kvstore:
        db_path: /home/jbarea/.llama/distributions/ollama/meta_reference_eval.db
        namespace: null
        type: sqlite
    provider_id: meta-reference
    provider_type: inline::meta-reference
  inference:
  - config:
      url: http://localhost:11434
    provider_id: ollama
    provider_type: remote::ollama
  post_training:
  - config:
      checkpoint_format: huggingface
      device: cpu
      distributed_backend: null
    provider_id: huggingface
    provider_type: inline::huggingface
  safety:
  - config:
      excluded_categories: []
    provider_id: llama-guard
    provider_type: inline::llama-guard
  scoring:
  - config: {}
    provider_id: basic
    provider_type: inline::basic
  - config: {}
    provider_id: llm-as-judge
    provider_type: inline::llm-as-judge
  - config:
      openai_api_key: '********'
    provider_id: braintrust
    provider_type: inline::braintrust
  telemetry:
  - config:
      service_name: ''
      sinks: sqlite
      sqlite_db_path: /home/jbarea/.llama/distributions/ollama/trace_store.db
    provider_id: meta-reference
    provider_type: inline::meta-reference
  tool_runtime:
  - config:
      api_key: '********'
      max_results: 3
    provider_id: brave-search
    provider_type: remote::brave-search
  - config:
      api_key: '********'
      max_results: 3
    provider_id: tavily-search
    provider_type: remote::tavily-search
  - config: {}
    provider_id: rag-runtime
    provider_type: inline::rag-runtime
  - config: {}
    provider_id: model-context-protocol
    provider_type: remote::model-context-protocol
  - config:
      api_key: '********'
    provider_id: wolfram-alpha
    provider_type: remote::wolfram-alpha
  vector_io:
  - config:
      kvstore:
        db_path: /home/jbarea/.llama/distributions/ollama/faiss_store.db
        namespace: null
        type: sqlite
    provider_id: faiss
    provider_type: inline::faiss
scoring_fns: []
server:
  auth: null
  host: null
  port: 8321
  quota: null
  tls_cafile: null
  tls_certfile: null
  tls_keyfile: null
shields: []
tool_groups:
- args: null
  mcp_endpoint: null
  provider_id: tavily-search
  toolgroup_id: builtin::websearch
- args: null
  mcp_endpoint: null
  provider_id: rag-runtime
  toolgroup_id: builtin::rag
- args: null
  mcp_endpoi

We can ask specify what exactly we need to the LLM

In [2]:


prompt = "Use the kube_command to execute `kubectl get pods -A`. Show me the output"
print("prompt>", prompt)

response = agent.create_turn(
  messages=[{"role": "user", "content": prompt}],
  session_id=session_id,
)

for log in AgentEventLogger().log(response):
    log.print()

prompt> Use the kube_command to execute `kubectl get pods -A`. Show me the output
inference> [kube_command(cmd='kubectl get pods -A', session_id='session_123')]
tool_execution> Tool:kube_command Args:{'cmd': 'kubectl get pods -A', 'session_id': 'session_123'}
tool_execution> Tool:kube_command Response:[TextContentItem(text='{\n  "success": true,\n  "result": "NAMESPACE     NAME                               READY   STATUS    RESTARTS         AGE\\nkube-system   coredns-6f6b679f8f-6z4gj           1/1     Running   13 (86d ago)     181d\\nkube-system   etcd-minikube                      1/1     Running   13 (86d ago)     181d\\nkube-system   kindnet-6jvrc                      1/1     Running   0                181d\\nkube-system   kindnet-jmlkf                      1/1     Running   14 (86d ago)     181d\\nkube-system   kindnet-k4g5p                      1/1     Running   0                181d\\nkube-system   kube-apiserver-minikube            1/1     Running   13 (86d ago)     181d\\nku

Or just executing a simple task leaves it to the LLM’s interpretation

In [3]:
prompt = "Create a new namespace called mcp-tests."
print("prompt>", prompt)

response = agent.create_turn(
  messages=[{"role": "user", "content": prompt}],
  session_id=session_id,
)

for log in AgentEventLogger().log(response):
    log.print()

prompt> Create a new namespace called mcp-tests.
inference> [kube_command(cmd="kubectl create namespace mcp-tests", session_id="session_123")]
tool_execution> Tool:kube_command Args:{'cmd': 'kubectl create namespace mcp-tests', 'session_id': 'session_123'}
tool_execution> Tool:kube_command Response:[TextContentItem(text='{\n  "success": true,\n  "result": "namespace/mcp-tests created\\n"\n}', type='text')]
inference> The namespace `mcp-tests` has been successfully created. 

```
Namespace:   mcp-tests
```


Even it's possible to ask complex task which imply differents steps.

In [4]:
prompt = "Deploy the hashicorp/http-echo:latest service in mcp-tests namespace. It should be accessible from outside cluster. Then use the yaml created and call the kube_cmd tool to create the resources in the cluster. I don't want the yaml I want you use the kube_cmd tool to create the resources. You are force to call the tool"

print("prompt>", prompt)

response = agent.create_turn(
  messages=[{"role": "user", "content": prompt}],
  session_id=session_id,
)

for log in AgentEventLogger().log(response):
    log.print()

prompt> Deploy the hashicorp/http-echo:latest service in mcp-tests namespace. It should be accessible from outside cluster. Then use the yaml created and call the kube_cmd tool to create the resources in the cluster. I don't want the yaml I want you use the kube_cmd tool to create the resources. You are force to call the tool
inference> [kube_command(cmd="kubectl create deployment http-echo -n mcp-tests --image=hashicorp/http-echo:latest", session_id="session_123")]
[kube_command(cmd="kubectl expose deployment http-echo -n mcp-tests -t 80:80", session_id="session_123")]
[kube_command(cmd="kubectl get svc http-echo -n mcp-tests --type=LoadBalancer", session_id="session_123")]
tool_execution> Tool:kube_command Args:{'cmd': 'kubectl create deployment http-echo -n mcp-tests --image=hashicorp/http-echo:latest', 'session_id': 'session_123'}
tool_execution> Tool:kube_command Response:[TextContentItem(text='{\n  "success": true,\n  "result": "deployment.apps/http-echo created\\n"\n}', type='te